### Indexing on ndarrays

`ndarrays` can be indexed using the standard Python `x[obj]` syntax, where x is the array and obj the selection. There are different kinds of indexing available depending on obj: basic indexing, advanced indexing and field access.

> [!NOTE]
> Note that in Python, `x[(exp1, exp2, ..., expN)]` is equivalent to `x[exp1, exp2, ..., expN]`; the latter is just syntactic sugar for the former.

### Basic indexing

#### Single element indexing
Single element indexing works exactly like that for other standard Python sequences


In [9]:
import numpy as np
x = np.arange(10)
print(x[2])
print(x[-2])

2
8


It is not necessary to separate each dimension’s index into its own set of square brackets.

In [10]:
x.shape = (2, 5) # now x is 2-d
print(x[1, 3])
print(x[1, -1])

8
9


> [!NOTE]
> Note that if one indexes a multidimensional array with fewer indices than dimensions, one gets a subdimensional array. For example:

In [11]:
print(x[0])

[0 1 2 3 4]


That is, each index specified selects the array corresponding to the rest of the dimensions selected. In the above example, choosing 0 means that the remaining dimension of length 5 is being left unspecified, and that what is returned is an array of that dimensionality and size.

In [15]:
c = x[0][2]
print(c)
is_equal = x[0, 2] == x[0][2]
print(is_equal)

0.9355409765922167
True


### Slicing and striding

Basic slicing extends Python’s basic concept of slicing to N dimensions. Basic slicing occurs when obj is a slice object (constructed by `start:stop:step` notation inside of brackets), an integer, or a tuple of slice objects and integers.
The basic slice syntax is `i:j:k` where i is the starting index, j is the stopping index, and k is the step (*k* != 0).

In [19]:
x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
print(x)
print(x[1:7:2])

[0 1 2 3 4 5 6 7 8 9]
[1 3 5]


In [20]:
print(x[-2: 10])
print(x[-3:3:-1])

[8 9]
[7 6 5 4]


Note that `::` is the same as `:` and means select all indices along this axis. From the above example:

In [21]:
print(x[5:])

[5 6 7 8 9]


In [24]:
x = np.array([[[1],[2],[3]], [[4],[5],[6]]])
print(x)
print(x.shape)
# 2d array, 3 rows, 1 column (2, 3, 1)
print(x[1:2])

[[[1]
  [2]
  [3]]

 [[4]
  [5]
  [6]]]
(2, 3, 1)
[[[4]
  [5]
  [6]]]


### Dimensional indexing tools

`Ellipsis` expands to the number of `:` objects needed for the selection tuple to index all dimensions. In most cases, this means that the length of the expanded selection tuple is `x.ndim`.

In [25]:
print(x[..., 0])

[[1 2 3]
 [4 5 6]]


This is equivalent to

In [26]:
print(x[:, :, 0])

[[1 2 3]
 [4 5 6]]


Each `newaxis` object in the selection tuple serves to expand the dimensions of the resulting selection by one unit-length dimension. The added dimension is the position of the `newaxis` object in the selection tuple. `newaxis` is an alias for None, and None can be used in place of this with the same result

In [28]:
print(x[:, np.newaxis, :, :].shape)
print(x[:, None, :, :].shape)

(2, 1, 3, 1)
(2, 1, 3, 1)


This can be handy to combine two arrays in a way that otherwise would require explicit reshaping operations

In [30]:
x = np.arange(5)
print(x)
print("-" * 20)
print(x[:, np.newaxis] + x[np.newaxis, :])

[0 1 2 3 4]
--------------------
[[0 1 2 3 4]
 [1 2 3 4 5]
 [2 3 4 5 6]
 [3 4 5 6 7]
 [4 5 6 7 8]]


### Advanced indexing
Advanced indexing is triggered when the selection object, obj, is a non-tuple sequence object, an `ndarray` (of data type integer or bool), or a tuple with at least one sequence object or ndarray (of data type integer or bool). There are two types of advanced indexing: integer and Boolean.

> [! WARNING]
> The definition of advanced indexing means that `x[(1, 2, 3),]` is fundamentally different than `x[(1, 2, 3)]`. The latter is equivalent to `x[1, 2, 3]` which will trigger basic selection while the former will trigger advanced indexing. Be sure to understand why this occurs.

### Integer array indexing
Integer array indexing allows selection of arbitrary items in the array based on their N-dimensional index

In [32]:
x = np.arange(10, 1, -1)
print(x)
print("-" * 20)
print(x[np.array([3, 3, 1, 8])])
print("-" * 20)
print(x[np.array([3, 3, -3, 8])])

[10  9  8  7  6  5  4  3  2]
--------------------
[7 7 9 2]
--------------------
[7 7 4 2]


If the index values are out of bounds then an _`IndexError`_ is thrown

In [31]:
x = np.array([[1, 2], [3, 4], [5, 6]])
print(x[np.array([1, -1])])

[[3 4]
 [5 6]]


Indexing with multidimensional index arrays tend to be more unusual uses, but they are permitted, and they are useful for some problems. We’ll start with the simplest multidimensional case:

In [35]:
a = np.arange(6).reshape((3, 2))
print(a)
# As you can see, it created a 3 x 2 array (Meaning a 3 row, 2 column,array)
print("-" * 20)
print(np.reshape(a, (2, 3)))

[[0 1]
 [2 3]
 [4 5]]
--------------------
[[0 1 2]
 [3 4 5]]


The reshape property, allows you to share the array into another order as shown in the example above.

In [33]:
y = np.arange(35).reshape(5, 7)
print(y)
print("-" * 20)
print(y[np.array([0, 2, 4]), np.array([0, 1, 2])])

[[ 0  1  2  3  4  5  6]
 [ 7  8  9 10 11 12 13]
 [14 15 16 17 18 19 20]
 [21 22 23 24 25 26 27]
 [28 29 30 31 32 33 34]]
--------------------
[ 0 15 30]


If the index arrays do not have the same shape, there is an attempt to broadcast them to the same shape. If they cannot be broadcast to the same shape, an exception is raised:



In [40]:
print(y[np.array([0, 2, 4]), np.array([0, 1])])

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (3,) (2,) 

The broadcasting mechanism permits index arrays to be combined with scalars for other indices. The effect is that the scalar value is used for all the corresponding values of the index arrays:

In [42]:
print(y[np.array([0, 2, 4])])

IndexError: index 2 is out of bounds for axis 0 with size 1

### Example
From each row, a specific element should be selected. The row index is just `[0, 1, 2]` and the column index specifies the element to choose for the corresponding row, here `[0, 1, 0]`. Using both together the task can be solved using advanced indexing:

In [46]:
x = np.array([[1, 2], [3, 4], [5, 6]])
print(x[[0, 1, 2], [0, 1, 0]])

[1 4 5]


### Example 2
rom a 4x3 array the corner elements should be selected using advanced indexing. Thus all elements for which the column is one of `[0, 2]` and the row is one of `[0, 3]` need to be selected. To use advanced indexing one needs to select all elements explicitly. Using the method explained previously one could write:

In [47]:
x = np.array([[ 0,  1,  2],
              [ 3,  4,  5],
              [ 6,  7,  8],
              [ 9, 10, 11]])
rows = np.array([[0, 0],
                 [3, 3]], dtype=np.intp)
columns = np.array([[0, 2],
                    [0, 2]], dtype=np.intp)
print(x[rows, columns])

[[ 0  2]
 [ 9 11]]


### Boolean array indexing
This advanced indexing occurs when obj is an array object of Boolean type, such as may be returned from comparison operators. A single boolean index array is practically identical to `x[obj.nonzero()]` where, as described above, `obj.nonzero()` returns a tuple (of length `obj.ndim`) of integer index arrays showing the True elements of _obj_.

In [48]:
x = np.array([[1., 2.], [np.nan, 3.], [np.nan, np.nan]])
print(x[~np.isnan(x)])

[1. 2. 3.]


Or wish to add a constant to all negative elements:

In [49]:
x = np.array([1., -1., -2., 3])
x[x < 0] += 20
print(x)

[ 1. 19. 18.  3.]


### Combining advanced and basic indexing
When there is at least one slice (`:`), ellipsis (`...`) or `newaxis` in the index (or the array has more dimensions than there are advanced indices), then the behaviour can be more complicated. It is like concatenating the indexing result for each advanced index element.


In [50]:
y = np.arange(35).reshape(5,7)
print(y[np.array([0, 2, 4]), 1:3])

[[ 1  2]
 [15 16]
 [29 30]]


In effect, the slice and index array operation are independent. The slice operation extracts columns with index 1 and 2, (i.e. the 2nd and 3rd columns), followed by the index array operation which extracts rows with index 0, 2 and 4 (i.e the first, third and fifth rows). This is equivalent to:

In [51]:
print(y[:, 1:3][np.array([0, 2, 4]), :])

[[ 1  2]
 [15 16]
 [29 30]]


A single advanced index can, for example, replace a slice and the result array will be the same. However, it is a copy and may have a different memory layout. A slice is preferable when it is possible. For example:

In [56]:
x = np.array([[ 0,  1,  2],
              [ 3,  4,  5],
              [ 6,  7,  8],
              [ 9, 10, 11]])
# row, column
print(x[1:2, 1:3])

[[4 5]]


### Field access
If the `ndarray` object is a structured array the fields of the array can be accessed by indexing the array with strings, dictionary-like.
Indexing `x['field-name']` returns a new view to the array, which is of the same shape as x (except when the field is a sub-array) but of data type `x.dtype['field-name']` and contains only the part of the data in the specified field. Also, record array scalars can be “indexed” this way.

### Flat iterator indexing
`x.flat` returns an iterator that will iterate over the entire array (in C-contiguous style with the last index varying the fastest). This iterator object can also be indexed using basic slicing or advanced indexing as long as the selection object is not a tuple. This should be clear from the fact that `x.flat` is a 1-dimensional view.

### Assigning values to indexed arrays
As mentioned, one can select a subset of an array to assign to using a single index, slices, and index and mask arrays. The value being assigned to the indexed array must be shape consistent (the same shape or broadcastable to the shape the index produces).

In [57]:
x = np.arange(10)
x[2:7] = 1
print(x)

[0 1 1 1 1 1 1 7 8 9]


or an array of the right size:

In [58]:
x[2:7] = np.arange(5)
print(x)

[0 1 0 1 2 3 4 7 8 9]


### Dealing with variable numbers of indices within programs
The indexing syntax is very powerful but limiting when dealing with a variable number of indices. For example, if you want to write a function that can handle arguments with various numbers of dimensions without having to write special case code for each number of possible dimensions, how can that be done?


In [59]:
z = np.arange(81).reshape(3, 3, 3, 3)
indices = (1, 1, 1, 1)
print(z[indices])

40


So one can use code to construct tuples of any number of indices and then use these within an index.

Slices can be specified within programs by using the slice() function in Python. For example

In [60]:
indices = (1, 1, 1, slice(0, 2))  # same as [1, 1, 1, 0:2]
print(z[indices])

[39 40]


Likewise, ellipsis can be specified by code by using the Ellipsis object:

In [63]:
indices = (1, Ellipsis, 1)  # same as [1, ..., 1]
print(z[indices])
print("-" * 20)
# performing the below returns a single value
print(z[(1, 1, 1, 1)])

[[28 31 34]
 [37 40 43]
 [46 49 52]]
--------------------
40


### Additional notes
- The native NumPy indexing type is `intp` and may differ from the default integer array type. intp is the smallest data type sufficient to safely index any array; for advanced indexing it may be faster than other types
- For advanced assignments, there is in general no guarantee for the iteration order. This means that if an element is set more than once, it is not possible to predict the final result.

An empty (tuple) index is a full scalar index into a zero-dimensional array. `x[()]` returns a scalar if x is zero-dimensional and a view otherwise. On the other hand, `x[...]` always returns a view.